In [1]:
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy import Column, Float, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import sqlite3 as db

%matplotlib inline

In [2]:
#Establish path to the CSV file that was created from the cleaning dataframe
IMDB_data_path = "../Project 2/clean_IMDB_data.csv"

In [3]:
#Create a new dataframe from the clean CSV
IMDB_data = pd.read_csv(IMDB_data_path)

IMDB_df = pd.DataFrame(IMDB_data)

del IMDB_df['Unnamed: 0']

IMDB_df

,AUTHOR,TITLE,REVIEW,RATING
0,margarida-44311,Not Bad,I don't get all the terrible reviews for this ...,5.0
1,joemay-2,What are all the bad reviews about is it a wo...,I cannot believe anyone could give this film l...,8.0
2,nebk,Great White=Jaws Lite,Great White is not the worst way to spend 90 m...,4.0
3,kuarinofu,Bare-bones killer shark film,Great White is as basic of a killer shark film...,4.0
4,Horror_Flick_Fanatic,"Terrible story, dialogue, and CGI","Terrible story, dialogue and CGI. The film has...",4.0
...,...,...,...,...
5327,suryajijvania,More Parts,"It's master piece by Zack please part 2,3,4 al...",10.0
5328,shishirkmr-82243,It's a fantastic movie,No words to describe. It's awesome. One of the...,10.0
5329,moizsyed-07601,Awesome out standing!,Far better than previous one and better editin...,10.0
5330,samun_shrestha,EPIC,Why did the studio say no to this masterpiece?...,10.0


In [4]:
#Create and author dataframe to load to the new sqlite db authors table.
author = pd.DataFrame(IMDB_df['AUTHOR'])

author

,AUTHOR
0,margarida-44311
1,joemay-2
2,nebk
3,kuarinofu
4,Horror_Flick_Fanatic
...,...
5327,suryajijvania
5328,shishirkmr-82243
5329,moizsyed-07601
5330,samun_shrestha


In [5]:
# Create dataframe for the Authors table and rename the index to be the author_id 
# ----------------------------------
author_df = author.drop_duplicates().reset_index(drop=True)
author_df = author_df.rename_axis("author_id")

author_df.rename(columns = {'AUTHOR':'author_name'}, inplace=True)

author_df

,author_name
author_id,
0,margarida-44311
1,joemay-2
2,nebk
3,kuarinofu
4,Horror_Flick_Fanatic
...,...
4673,suryajijvania
4674,shishirkmr-82243
4675,moizsyed-07601


In [6]:
# Create dataframe for the movies table
movie_df = pd.DataFrame(IMDB_df[['TITLE', 'REVIEW', 'RATING', 'AUTHOR']])
movie_df = movie_df.rename_axis("movie_id")
movie_df.rename(columns = {'TITLE': 'movie_title', 'REVIEW':'movie_review', 'RATING':'movie_rating', 'AUTHOR':'author_name'}, inplace=True)

movie_df

,movie_title,movie_review,movie_rating,author_name
movie_id,,,,
0,Not Bad,I don't get all the terrible reviews for this ...,5.0,margarida-44311
1,What are all the bad reviews about is it a wo...,I cannot believe anyone could give this film l...,8.0,joemay-2
2,Great White=Jaws Lite,Great White is not the worst way to spend 90 m...,4.0,nebk
3,Bare-bones killer shark film,Great White is as basic of a killer shark film...,4.0,kuarinofu
4,"Terrible story, dialogue, and CGI","Terrible story, dialogue and CGI. The film has...",4.0,Horror_Flick_Fanatic
...,...,...,...,...
5327,More Parts,"It's master piece by Zack please part 2,3,4 al...",10.0,suryajijvania
5328,It's a fantastic movie,No words to describe. It's awesome. One of the...,10.0,shishirkmr-82243
5329,Awesome out standing!,Far better than previous one and better editin...,10.0,moizsyed-07601


In [7]:
# Create engine for establishing the new sqlite db
# ----------------------------------
engine = create_engine(f"sqlite:///IMDB2.sqlite")
conn = engine.connect()

In [8]:
#Use engine.execute to create the table schema and set primary key
engine.execute('CREATE TABLE "authors" ('
               'author_id INTEGER NOT NULL PRIMARY KEY,'
               'author_name VARCHAR);')
               

In [9]:
#Load the data from the authors dataframe to the sqlite authors table
author_df.to_sql(name='authors', con=conn, if_exists='append', index=True, index_label="author_id")

In [10]:
# read database and create authors dataframe to merge with the movies dataframe
# ----------------------------------
authors = pd.read_sql('select * from authors', conn)

authors

,author_id,author_name
0,0,margarida-44311
1,1,joemay-2
2,2,nebk
3,3,kuarinofu
4,4,Horror_Flick_Fanatic
...,...,...
4673,4673,suryajijvania
4674,4674,shishirkmr-82243
4675,4675,moizsyed-07601
4676,4676,samun_shrestha


In [11]:
#merge dataframes to create movie table to prepare to populate the movie sqlite table and use author_id as the link between the authors 
#and movie tables
combined_table = pd.merge(authors, movie_df, on="author_name")

combined_table

,author_id,author_name,movie_title,movie_review,movie_rating
0,0,margarida-44311,Not Bad,I don't get all the terrible reviews for this ...,5.0
1,1,joemay-2,What are all the bad reviews about is it a wo...,I cannot believe anyone could give this film l...,8.0
2,2,nebk,Great White=Jaws Lite,Great White is not the worst way to spend 90 m...,4.0
3,3,kuarinofu,Bare-bones killer shark film,Great White is as basic of a killer shark film...,4.0
4,3,kuarinofu,Quite a good sci-fi film,The plot itself is not unique and is rather a ...,7.0
...,...,...,...,...,...
5327,4675,moizsyed-07601,Awesome out standing!,Far better than previous one and better editin...,10.0
5328,4676,samun_shrestha,EPIC,Why did the studio say no to this masterpiece?...,10.0
5329,4676,samun_shrestha,EPIC,Why did the studio say no to this masterpiece?...,10.0
5330,4677,mmuradali-65680,The best DC movie till date,Overall Opinion-Although the competitors Marve...,10.0


In [12]:
#Create a new dataframe to drop the author_name and set the index as the movie_id
combined_table2 = pd.DataFrame(combined_table, columns=['author_id', 'movie_title', 'movie_review', 'movie_rating'])
combined_table2 = combined_table2.rename_axis("movie_id")                               
combined_table2                               

,author_id,movie_title,movie_review,movie_rating
movie_id,,,,
0,0,Not Bad,I don't get all the terrible reviews for this ...,5.0
1,1,What are all the bad reviews about is it a wo...,I cannot believe anyone could give this film l...,8.0
2,2,Great White=Jaws Lite,Great White is not the worst way to spend 90 m...,4.0
3,3,Bare-bones killer shark film,Great White is as basic of a killer shark film...,4.0
4,3,Quite a good sci-fi film,The plot itself is not unique and is rather a ...,7.0
...,...,...,...,...
5327,4675,Awesome out standing!,Far better than previous one and better editin...,10.0
5328,4676,EPIC,Why did the studio say no to this masterpiece?...,10.0
5329,4676,EPIC,Why did the studio say no to this masterpiece?...,10.0


In [13]:
#Use engine.execute to create the table schema and set primary key
engine.execute('CREATE TABLE "movies" ('
               'movie_id INTEGER NOT NULL PRIMARY KEY,'
               'author_id INTEGER NOT NULL,'
               'movie_title VARCHAR,'
               'movie_review VARCHAR,'
            'movie_rating FLOAT);')

In [14]:
#Load the dataframe data to the sqlite db movies table
combined_table2.to_sql(name='movies', con=conn, if_exists='append', index=True, index_label="movie_id")

In [15]:
# read database to create a new dataframe that includes the author_id
# ----------------------------------
movies = pd.read_sql('select * from movies', conn)

movies

,movie_id,author_id,movie_title,movie_review,movie_rating
0,0,0,Not Bad,I don't get all the terrible reviews for this ...,5.0
1,1,1,What are all the bad reviews about is it a wo...,I cannot believe anyone could give this film l...,8.0
2,2,2,Great White=Jaws Lite,Great White is not the worst way to spend 90 m...,4.0
3,3,3,Bare-bones killer shark film,Great White is as basic of a killer shark film...,4.0
4,4,3,Quite a good sci-fi film,The plot itself is not unique and is rather a ...,7.0
...,...,...,...,...,...
5327,5327,4675,Awesome out standing!,Far better than previous one and better editin...,10.0
5328,5328,4676,EPIC,Why did the studio say no to this masterpiece?...,10.0
5329,5329,4676,EPIC,Why did the studio say no to this masterpiece?...,10.0
5330,5330,4677,The best DC movie till date,Overall Opinion-Although the competitors Marve...,10.0


In [16]:
#Query to aggregat rating statistics by author based on dataframe from the original database
auth_agg_df2 = IMDB_df.groupby(['AUTHOR'])

count = auth_agg_df2["RATING"].count()
rating_mean = auth_agg_df2["RATING"].mean()
rating_median = auth_agg_df2["RATING"].median()
rating_min = auth_agg_df2["RATING"].min()
rating_max = auth_agg_df2['RATING'].max()

aggregate_df = pd.DataFrame({"Rating Count": count, "Rating Mean": rating_mean, "Rating Median": rating_median, "Rating Min": rating_min,
                             "Rating Max": rating_max})

auth_agg = aggregate_df.sort_values(['Rating Count'], ascending=False)

auth_agg.head(50)

,Rating Count,Rating Mean,Rating Median,Rating Min,Rating Max
AUTHOR,,,,,
Xstal,10,6.100000,7.0,3.0,9.0
kwenchow,10,1.500000,1.0,1.0,6.0
yusufpiskin,9,6.888889,8.0,3.0,9.0
cruise01,9,5.333333,5.0,2.0,8.0
nogodnomasters,9,5.777778,5.0,3.0,10.0
ops-52535,9,5.444444,6.0,2.0,10.0
SnoopyStyle,8,5.125000,5.0,3.0,8.0
Golden_Hope,8,5.500000,5.5,4.0,7.0
ThomDerd,8,6.000000,6.0,3.0,9.0


In [17]:
#Query to aggregate author rating statistics based on dataframe created from the new sqlite db to include the author ID.
auth_agg_df3 = combined_table.groupby(['author_name', 'author_id'])

count = auth_agg_df3["movie_rating"].count()
rating_mean = auth_agg_df3["movie_rating"].mean()
rating_median = auth_agg_df3["movie_rating"].median()
rating_min = auth_agg_df3["movie_rating"].min()
rating_max = auth_agg_df3['movie_rating'].max()

aggregate_df = pd.DataFrame({"Rating Count": count, "Rating Mean": rating_mean, "Rating Median": rating_median, "Rating Min": rating_min,
                             "Rating Max": rating_max})

auth_agg2 = aggregate_df.sort_values(['Rating Count'], ascending=False)

auth_agg2.head(50)

,,Rating Count,Rating Mean,Rating Median,Rating Min,Rating Max
author_name,author_id,,,,,
Xstal,49,10,6.100000,7.0,3.0,9.0
kwenchow,485,10,1.500000,1.0,1.0,6.0
yusufpiskin,976,9,6.888889,8.0,3.0,9.0
cruise01,824,9,5.333333,5.0,2.0,8.0
nogodnomasters,28,9,5.777778,5.0,3.0,10.0
ops-52535,10,9,5.444444,6.0,2.0,10.0
SnoopyStyle,87,8,5.125000,5.0,3.0,8.0
Golden_Hope,170,8,5.500000,5.5,4.0,7.0
ThomDerd,221,8,6.000000,6.0,3.0,9.0


In [18]:
auth_agg3 = auth_agg2[(auth_agg2['Rating Count'] >=5) & (auth_agg2['Rating Mean'] >=7)]

auth_agg3 = auth_agg3.sort_values(['Rating Mean'], ascending=False)

auth_agg3

,,Rating Count,Rating Mean,Rating Median,Rating Min,Rating Max
author_name,author_id,,,,,
RosanaBotafogo,1171,5,8.200000,8.0,8.0,9.0
jadepietro,778,5,8.000000,8.0,7.0,10.0
masonsaul,171,5,7.400000,7.0,6.0,9.0
joscon-16616,113,7,7.142857,7.0,6.0,8.0
masonfisk,1180,5,7.000000,7.0,6.0,8.0
